# **IATI Datastore API data collection**

## **Dependencies**

In [1]:
import requests
import csv
from tqdm import tqdm
import KEYS
import json

## **Define relevant features**

List of all elements found here: https://developer.iatistandard.org/api-details#api=datastore&operation=query 

Description of all elements found here: https://iatistandard.org/en/iati-standard/203/activity-standard/summary-table/

In [3]:
fl_list = [
    "iati_identifier",
    "title_narrative",
    "title_narrative_xml_lang",
    "reporting_org_narrative",
    "description_narrative_xml_lang",
    "description_narrative",
    "recipient_country_code",
    "recipient_region_code",
    "location_name_narrative",
    "activity_status_code",
    "document_link_url",
    "last_updated_datetime",
    "activity_date_type",
    "activity_date_iso_date",
    "activity_date_narrative",
    "sector_code",
    "sector_narrative",
    "sector_vocabulary",
    "tag_code",
    "tag_narrative",
    "tag_vocabulary",
    "policy_marker_code",
    "policy_marker_narrative",
    "policy_marker_code",
    "default_aid_type_code"
]

fl_str = ",".join(fl_list)

## **Fetch data from IATI Datastore**

## Fetch as JSON

In [25]:
max_pages = 100

url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 0, 
    "rows": 1000, 
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY2
}

try:
    all_docs = []  
    page = 0
    while True:
        page +=1 
        response = requests.get(url, params=query, headers=headers)
        if page < max_pages:
            if response.status_code == 200:
                data = response.json()
                docs = data.get("response", {}).get("docs", [])
                if docs:
                    all_docs.extend(docs)  
                    query["start"] += len(docs)  
                else:
                    break  
            else:
                print("Error:", response.status_code)
                break
        else:
            break


    with open("../../src/response_iati.json", mode='w', encoding='utf-8') as file:
        json.dump(all_docs, file, ensure_ascii=False, indent=4)
    print("Data downloaded and stored as 'response_iati.json'")

except requests.exceptions.RequestException as e:
    print("Error:", e)

Data downloaded and stored as 'response_iati.json'


## Fetch as CSV

In [58]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 500000,
    "rows": 1000,
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY2
}

try:
    response = requests.get(url, params=query, headers=headers)
    if response.status_code == 200:
        data = response.json()
        docs = data.get("response", {}).get("docs", [])
        if docs:
            with open("../data/response_iati.csv", mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fl_list)
                writer.writeheader()
                for doc in docs:
                    row = {key: doc.get(key, '') for key in fl_list}
                    writer.writerow(row)
            print("Data downlaoded and stored as 'response_iati.csv'")
        else:
            print("No data in response!")
    else:
        print("Error:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Error:", e)

Data downlaoded and stored as 'response_iati.csv'
